In [111]:
import pandas as pd

In [112]:
business = pd.read_csv('./data/business.csv')
#atributtes = pd.read_csv('./data/attributes.csv', low_memory=False)
#categories = pd.read_csv('./data/categories.csv')
hours = pd.read_csv('./hours_fixed.csv')
gps = pd.read_csv('./data/gps_clustering.csv')
checkins = pd.read_csv('./data/n_checkins.csv')
top_tips = pd.read_csv('./data/top_tips.csv')
metrics = pd.read_csv('./data/crude_metrics.csv')
target = pd.read_csv('./data/target.csv')
business2 = pd.read_json('./data/business.json', lines = True)

In [113]:
business.head()

,business_id,name,postal_code,latitude,longitude,stars,review_count,is_open
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ",93101,34.426679,-119.711197,5.0,7,0
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,63123,38.551126,-90.335695,3.0,15,1
2,tUFrWirKiKi_TAnsVWINQQ,Target,85711,32.223236,-110.880452,3.5,22,0
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,19107,39.955505,-75.155564,4.0,80,1
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,18054,40.338183,-75.471659,4.5,13,1


In [114]:
atributtes = pd.json_normalize(data = business2['attributes'])

In [115]:
atributtes

,ByAppointmentOnly,BusinessAcceptsCreditCards,BikeParking,RestaurantsPriceRange2,CoatCheck,RestaurantsTakeOut,RestaurantsDelivery,Caters,WiFi,BusinessParking,...,AcceptsInsurance,BestNights,BYOB,Corkage,BYOBCorkage,HairSpecializesIn,Open24Hours,RestaurantsCounterService,AgesAllowed,DietaryRestrictions
0,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,False,True,True,2,False,False,False,False,u'no',"{'garage': False, 'street': False, 'validated'...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,False,False,True,1,NaN,True,False,True,u'free',"{'garage': False, 'street': True, 'validated':...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,True,True,NaN,NaN,True,NaN,False,NaN,"{'garage': None, 'street': None, 'validated': ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150341,False,NaN,NaN,3,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
150342,NaN,True,True,2,NaN,NaN,NaN,NaN,u'no',"{'garage': False, 'street': False, 'validated'...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
150343,NaN,True,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
150344,NaN,True,True,4,NaN,None,None,NaN,NaN,"{'garage': False, 'street': False, 'validated'...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [116]:
atributtes['BusinessParking'].fillna("{'garage': False, 'street': False, 'validated': False, 'lot': False, 'valet': False}", inplace=True)
atributtes.loc[atributtes['BusinessParking'] == 'None', 'BusinessParking'] = "{'garage': False, 'street': False, 'validated': False, 'lot': False, 'valet': False}"

atributtes.loc[atributtes['Ambience'] == 'None', 'Ambience'] = "{'romantic': False, 'intimate': False, 'classy': False, 'hipster': False, 'touristy': False, 'trendy': False, 'upscale': False, 'casual': False}"
atributtes['Ambience'] = atributtes['Ambience'].fillna("{'romantic': False, 'intimate': False, 'classy': False, 'hipster': False, 'touristy': False, 'trendy': False, 'upscale': False, 'casual': False}")

In [117]:
import json
import ast

def dicc(row):
    #result = json.loads(row)
    result = ast.literal_eval(row)
    return result

In [118]:
garage = pd.json_normalize(atributtes.BusinessParking.apply(dicc))
ambience = pd.json_normalize(atributtes['Ambience'].apply(dicc))

In [119]:
garage.head()

,garage,street,validated,lot,valet
0,False,False,False,False,False
1,False,False,False,False,False
2,False,False,False,True,False
3,False,True,False,False,False
4,None,None,None,True,False


In [120]:
atributtes['garage'] = garage['garage']
atributtes['garage'].fillna(0, inplace=True)

In [121]:
atributtes.drop(['BusinessParking', 'Ambience'], axis=1, inplace=True)

In [122]:
ambience['good_ambience'] = ambience['romantic'] + ambience['intimate'] + ambience['classy'] + ambience['hipster'] + ambience['touristy'] + ambience['trendy'] + ambience['upscale'] + ambience['casual'] + ambience['divey']

In [123]:
ambience.loc[ambience['good_ambience'] > 1, 'good_ambience'] = 1
ambience.fillna(0, inplace=True)

In [124]:
ambience['good_ambience'].value_counts()

0    132647
1     17699
Name: good_ambience, dtype: int64

In [125]:
atributtes['good_ambience'] = ambience['good_ambience']

In [126]:
atributtes.head()

,ByAppointmentOnly,BusinessAcceptsCreditCards,BikeParking,RestaurantsPriceRange2,CoatCheck,RestaurantsTakeOut,RestaurantsDelivery,Caters,WiFi,WheelchairAccessible,...,BYOB,Corkage,BYOBCorkage,HairSpecializesIn,Open24Hours,RestaurantsCounterService,AgesAllowed,DietaryRestrictions,garage,good_ambience
0,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,0
1,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,0
2,False,True,True,2,False,False,False,False,u'no',True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,0
3,False,False,True,1,NaN,True,False,True,u'free',NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,0
4,NaN,True,True,NaN,NaN,True,NaN,False,NaN,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0


In [127]:
atributtes['delivery'] = atributtes['RestaurantsTakeOut'] + atributtes['RestaurantsDelivery']

In [128]:
atributtes.notna().sum().sort_values(ascending=False).head(20)

good_ambience                 150346
garage                        150346
BusinessAcceptsCreditCards    119765
RestaurantsPriceRange2         85314
BikeParking                    72638
RestaurantsTakeOut             59857
WiFi                           56914
RestaurantsDelivery            56282
delivery                       54628
GoodForKids                    53375
OutdoorSeating                 48802
RestaurantsReservations        45247
HasTV                          45084
RestaurantsGoodForGroups       44170
Alcohol                        43189
ByAppointmentOnly              42339
Caters                         40127
RestaurantsAttire              39255
NoiseLevel                     37993
GoodForMeal                    29087
dtype: int64

In [129]:
top20 = atributtes.notna().sum().sort_values(ascending=False).head(20).index.tolist()

In [130]:
atributtes[top20].head()

,good_ambience,garage,BusinessAcceptsCreditCards,RestaurantsPriceRange2,BikeParking,RestaurantsTakeOut,WiFi,RestaurantsDelivery,delivery,GoodForKids,OutdoorSeating,RestaurantsReservations,HasTV,RestaurantsGoodForGroups,Alcohol,ByAppointmentOnly,Caters,RestaurantsAttire,NoiseLevel,GoodForMeal
0,0,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN
1,0,False,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,False,True,2,True,False,u'no',False,FalseFalse,NaN,False,False,False,NaN,NaN,False,False,NaN,NaN,NaN
3,0,False,False,1,True,True,u'free',False,TrueFalse,NaN,False,NaN,NaN,NaN,u'none',False,True,NaN,NaN,NaN
4,0,0,True,NaN,True,True,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN


In [131]:
atributtes = atributtes[top20]

In [132]:
atributtes.fillna(0, inplace=True)
atributtes.replace('None', 0, inplace=True)
atributtes.replace('False', 0, inplace=True)
atributtes.replace(False, 0, inplace=True)
atributtes.replace('True', 1, inplace=True)

/tmp/ipykernel_32/1740209834.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  atributtes.fillna(0, inplace=True)
/tmp/ipykernel_32/1740209834.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  atributtes.replace('None', 0, inplace=True)
/tmp/ipykernel_32/1740209834.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  atributtes.replace('False', 0, inplace=True)
/tmp/ipykernel_32/1740209834.py:4: SettingWithCopyWarning: 
A value is t

In [133]:
atributtes['delivery'] = atributtes['RestaurantsTakeOut'] + atributtes['RestaurantsDelivery']
atributtes.drop(['RestaurantsTakeOut', 'RestaurantsDelivery'], axis=1, inplace=True)

/tmp/ipykernel_32/623812491.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  atributtes['delivery'] = atributtes['RestaurantsTakeOut'] + atributtes['RestaurantsDelivery']
/tmp/ipykernel_32/623812491.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  atributtes.drop(['RestaurantsTakeOut', 'RestaurantsDelivery'], axis=1, inplace=True)


In [134]:
import numpy as np
atributtes['WiFi'] = np.where(atributtes['WiFi'] != 0, 1, 0)

/tmp/ipykernel_32/659819437.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  atributtes['WiFi'] = np.where(atributtes['WiFi'] != 0, 1, 0)


In [140]:
atributtes.loc[atributtes['Alcohol'] == "u'none'", 'Alcohol'] = 0
atributtes.loc[atributtes['Alcohol'] == "'none'", 'Alcohol'] = 0
atributtes.loc[atributtes['Alcohol'] != 0, 'Alcohol'] = 1
atributtes['Alcohol'].unique()

array([0, 1], dtype=object)

In [173]:
atributtes.dtypes == 'object'

good_ambience                 False
garage                         True
BusinessAcceptsCreditCards    False
RestaurantsPriceRange2         True
BikeParking                   False
WiFi                          False
delivery                      False
GoodForKids                   False
OutdoorSeating                False
RestaurantsReservations       False
HasTV                         False
RestaurantsGoodForGroups      False
Alcohol                        True
ByAppointmentOnly             False
Caters                        False
RestaurantsAttire              True
NoiseLevel                     True
GoodForMeal                    True
dtype: bool

In [176]:
atributtes.loc[:,atributtes.dtypes == 'object'].sample(10)

,garage,RestaurantsPriceRange2,Alcohol,RestaurantsAttire,NoiseLevel,GoodForMeal
96113,0,1,0,0,0,0
45258,0,0,0,0,0,0
70874,0,0,0,0,0,0
142443,0,0,0,0,0,0
86802,0,0,0,0,0,0
117205,0,2,0,0,0,0
48236,0,2,1,1,2,"{'dessert': False, 'latenight': False, 'lunch'..."
30574,0,2,0,0,0,0
145223,0,1,0,1,2,0
131223,0,2,0,0,0,0


In [164]:
atributtes.loc[atributtes['NoiseLevel'].str.contains('quiet').fillna(False), 'NoiseLevel'] = 1
atributtes.loc[atributtes['NoiseLevel'].str.contains('average').fillna(False), 'NoiseLevel'] = 2
atributtes.loc[atributtes['NoiseLevel'].str.contains('loud').fillna(False), 'NoiseLevel'] = 3

In [167]:
atributtes.loc[atributtes['RestaurantsAttire'] != 0,'RestaurantsAttire'] = 1

In [175]:
atributtes.loc[atributtes['garage'] != 0,'garage'] = 1

In [192]:
atributtes.loc[atributtes['GoodForMeal'] == 0,'GoodForMeal'] = "{'dessert': False, 'latenight': False, 'lunch': False, 'dinner': False, 'brunch': False, 'breakfast': False}"
atributtes['GoodForMeal'].fillna("{'dessert': False, 'latenight': False, 'lunch': False, 'dinner': False, 'brunch': False, 'breakfast': False}", inplace=True)

/tmp/ipykernel_32/1790493757.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  atributtes['GoodForMeal'].fillna("{'dessert': False, 'latenight': False, 'lunch': False, 'dinner': False, 'brunch': False, 'breakfast': False}", inplace=True)


In [195]:
meal_types = pd.json_normalize(atributtes['GoodForMeal'].apply(dicc))

In [199]:
meal_types.describe()

,dessert,latenight,lunch,dinner,brunch,breakfast
count,145753,146602,147951,148994,146748,147887
unique,2,2,2,2,2,2
top,False,False,False,False,False,False
freq,143975,145271,133735,135896,143872,144855


In [ ]:
atributtes.drop(['GoodForMeal'], axis=1, inplace=True)
atributtes['meal_diversity'] = meal_types['dessert'] + meal_types['latenight'] + meal_types['lunch'] + meal_types['dinner'] + meal_types['brunch'] + meal_types['breakfast']

In [204]:
atributtes['meal_diversity'].fillna(0, inplace=True)

/tmp/ipykernel_32/55354110.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  atributtes['meal_diversity'].fillna(0, inplace=True)


In [205]:
atributtes.isna().sum().sort_values(ascending=False).head(20)

good_ambience                 0
garage                        0
NoiseLevel                    0
RestaurantsAttire             0
Caters                        0
ByAppointmentOnly             0
Alcohol                       0
RestaurantsGoodForGroups      0
HasTV                         0
RestaurantsReservations       0
OutdoorSeating                0
GoodForKids                   0
delivery                      0
WiFi                          0
BikeParking                   0
RestaurantsPriceRange2        0
BusinessAcceptsCreditCards    0
meal_diversity                0
dtype: int64

In [207]:
atributtes['business_id'] = business2['business_id']
atributtes.to_csv('./data/atributtes_clean.csv', index=False)

/tmp/ipykernel_32/4136999.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  atributtes['business_id'] = business2['business_id']


In [304]:
hours.sample(5)

,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday,business_id
98069,7:0-19:0,7:0-19:0,7:0-19:0,7:0-19:0,7:0-19:0,7:0-17:0,NaN,S5VCmo_NWpdYneAE0p93PA
95472,0:0-0:0,8:0-18:0,8:0-18:0,8:0-18:0,8:0-18:0,9:0-17:0,NaN,oS5I81wVdiD_yBNALMj64w
117543,9:0-18:0,9:0-18:0,9:0-19:0,9:0-18:0,9:0-18:0,8:30-13:0,NaN,Mro39taLejqtvsitvMCfGg
54846,8:0-17:0,8:0-17:0,8:0-17:0,8:0-17:0,8:0-17:0,8:0-15:0,NaN,YURjQmTSH2Nci8tmegOMbg
25790,0:0-0:0,0:0-0:0,0:0-0:0,0:0-0:0,0:0-0:0,0:0-0:0,0:0-0:0,IpfGcFwwCAh9Si_kjsoPhg


In [305]:
subset = hours[~pd.isnull(hours[['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']]).any(1)].index
hours['7days'] = 0
hours.loc[subset, '7days'] = 1

In [306]:
subset = hours[~pd.isnull(hours[['Friday', 'Saturday', 'Sunday']]).any(1)].index
hours['weekends'] = 0
hours.loc[subset, 'weekends'] = 1

In [307]:
def open_time(x):
    if pd.isnull(x):
        return None
    else:
        return x.split('-')[0]

def close_time(x):
    if pd.isnull(x):
        return None
    else:
        return x.split('-')[1]

In [308]:
hours['monday_open_time'] = pd.to_datetime(hours['Monday'].apply(open_time), format='%H:%M')
hours['monday_close_time'] = pd.to_datetime(hours['Monday'].apply(close_time), format='%H:%M')
hours['monday_total_hours'] = (hours['monday_close_time'] - hours['monday_open_time'])/pd.Timedelta(hours=1)
hours.loc[hours['monday_total_hours'] == 0.0, 'monday_total_hours'] = 24.0

hours['tuesday_open_time'] = pd.to_datetime(hours['Tuesday'].apply(open_time), format='%H:%M')
hours['tuesday_close_time'] = pd.to_datetime(hours['Tuesday'].apply(close_time), format='%H:%M')
hours['tuesday_total_hours'] = (hours['tuesday_close_time'] - hours['tuesday_open_time'])/pd.Timedelta(hours=1)
hours.loc[hours['tuesday_total_hours'] == 0.0, 'tuesday_total_hours'] = 24.0


hours['wednesday_open_time'] = pd.to_datetime(hours['Wednesday'].apply(open_time), format='%H:%M')
hours['wednesday_close_time'] = pd.to_datetime(hours['Wednesday'].apply(close_time), format='%H:%M')
hours['wednesday_total_hours'] = (hours['wednesday_close_time'] - hours['wednesday_open_time'])/pd.Timedelta(hours=1)
hours.loc[hours['wednesday_total_hours'] == 0.0, 'wednesday_total_hours'] = 24.0


hours['thursday_open_time'] = pd.to_datetime(hours['Thursday'].apply(open_time), format='%H:%M')
hours['thursday_close_time'] = pd.to_datetime(hours['Thursday'].apply(close_time), format='%H:%M')
hours['thursday_total_hours'] = (hours['thursday_close_time'] - hours['thursday_open_time'])/pd.Timedelta(hours=1)
hours.loc[hours['thursday_total_hours'] == 0.0, 'thursday_total_hours'] = 24.0


hours['friday_open_time'] = pd.to_datetime(hours['Friday'].apply(open_time), format='%H:%M')
hours['friday_close_time'] = pd.to_datetime(hours['Friday'].apply(close_time), format='%H:%M')
hours['friday_total_hours'] = (hours['friday_close_time'] - hours['friday_open_time'])/pd.Timedelta(hours=1)
hours.loc[hours['friday_total_hours'] == 0.0, 'friday_total_hours'] = 24.0


hours['saturday_open_time'] = pd.to_datetime(hours['Saturday'].apply(open_time), format='%H:%M')
hours['saturday_close_time'] = pd.to_datetime(hours['Saturday'].apply(close_time), format='%H:%M')
hours['saturday_total_hours'] = (hours['saturday_close_time'] - hours['saturday_open_time'])/pd.Timedelta(hours=1)
hours.loc[hours['saturday_total_hours'] == 0.0, 'saturday_total_hours'] = 24.0


hours['sunday_open_time'] = pd.to_datetime(hours['Sunday'].apply(open_time), format='%H:%M')
hours['sunday_close_time'] = pd.to_datetime(hours['Sunday'].apply(close_time), format='%H:%M')
hours['sunday_total_hours'] = (hours['sunday_close_time'] - hours['sunday_open_time'])/pd.Timedelta(hours=1)
hours.loc[hours['sunday_total_hours'] == 0.0, 'sunday_total_hours'] = 24.0


In [309]:
lista_total_open_hours = ['monday_total_hours', 'tuesday_total_hours', 'wednesday_total_hours', 'thursday_total_hours', 'friday_total_hours', 'saturday_total_hours', 'sunday_total_hours']
lista_open = ['monday_open_time', 'tuesday_open_time', 'wednesday_open_time', 'thursday_open_time', 'friday_open_time', 'saturday_open_time', 'sunday_open_time']
lista_close = ['monday_close_time', 'tuesday_close_time', 'wednesday_close_time', 'thursday_close_time', 'friday_close_time', 'saturday_close_time', 'sunday_close_time']
lista_days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
lista_total =  lista_total_open_hours + lista_open + lista_close + lista_days

In [310]:
hours['n_open_days'] = pd.notnull(hours[lista_open]).sum(axis=1)

In [311]:
hours['mean_total_hours_open'] = hours[lista_total_open_hours].abs().mean(axis=1)

In [312]:
hours['mean_total_hours_open'].describe()

count    127123.000000
mean         10.772699
std           4.154365
min           0.016667
25%           8.428571
50%          10.000000
75%          11.857143
max          24.000000
Name: mean_total_hours_open, dtype: float64

In [313]:
import datetime

def normalize_hours(lista:list, df):
    for i in lista:
        df.loc[pd.notnull(df[i]), i] = pd.to_datetime(df.loc[pd.notnull(df[i]), i]) - pd.to_datetime(df.loc[pd.notnull(df[i]), i]).dt.normalize()

def mean_open_hour(lista:list, df):

    for i in range(df.shape[0]):
        temp_sum = []
        for j in lista:
            if pd.notnull(df.loc[i, j]):
                temp_sum.append(df.loc[i, j])
        if len(temp_sum) > 0:
            df.loc[i, 'mean_open_hour'] = sum(temp_sum,datetime.timedelta())/len(temp_sum)

def mean_close_hour(lista:list, df):
    for i in range(df.shape[0]):
        temp_sum = []
        for j in lista:
            if pd.notnull(df.loc[i, j]):
                temp_sum.append(df.loc[i, j])
        if len(temp_sum) > 0:
            df.loc[i, 'mean_close_hour'] = sum(temp_sum,datetime.timedelta())/len(temp_sum)

In [314]:
normalize_hours(lista_open, hours)
normalize_hours(lista_close, hours)
mean_open_hour(lista_open, hours)
mean_close_hour(lista_close, hours)

In [315]:
hours.drop(lista_total, axis=1, inplace=True)

In [316]:
hours.head()

,business_id,7days,weekends,n_open_days,mean_total_hours_open,mean_open_hour,mean_close_hour
0,Pns2l4eNsfO8kk83dixA6A,0,0,0,NaN,NaT,NaT
1,mpf3x-BjTdTEA3yCZrAYPw,0,0,6,12.000000,0 days 06:40:00,0 days 14:40:00
2,tUFrWirKiKi_TAnsVWINQQ,1,1,7,14.285714,0 days 08:00:00,0 days 22:17:08.571428571
3,MTSW4McQd7CbVtyjqoe9mw,1,1,7,13.428571,0 days 07:00:00,0 days 20:25:42.857142857
4,mWMc6_wTdE0EUBKIGXDVfA,0,1,5,8.000000,0 days 13:12:00,0 days 21:12:00


In [317]:
hours.describe()

,7days,weekends,n_open_days,mean_total_hours_open,mean_open_hour,mean_close_hour
count,150346.000000,150346.000000,150346.000000,127123.000000,127123,127123
mean,0.474146,0.534115,5.327811,10.772699,0 days 08:52:15.181635109,0 days 16:06:55.853130994
std,0.499333,0.498836,2.445235,4.154365,0 days 03:14:14.536837022,0 days 06:06:27.727208463
min,0.000000,0.000000,0.000000,0.016667,0 days 00:00:00,0 days 00:00:00
25%,0.000000,0.000000,5.000000,8.428571,0 days 07:30:00,0 days 15:25:42.857142857
50%,0.000000,1.000000,6.000000,10.000000,0 days 09:00:00,0 days 17:36:00
75%,1.000000,1.000000,7.000000,11.857143,0 days 10:30:00,0 days 20:00:00
max,1.000000,1.000000,7.000000,24.000000,0 days 23:30:00,0 days 23:59:00


In [318]:
hours.to_csv('hours_fixed.csv', index=False)

In [9]:
## 6 == CANADA
gps.head()

,latitude,longitude,business_id,areas
0,34.426679,-119.711197,Pns2l4eNsfO8kk83dixA6A,0
1,38.551126,-90.335695,mpf3x-BjTdTEA3yCZrAYPw,2
2,32.223236,-110.880452,tUFrWirKiKi_TAnsVWINQQ,4
3,39.955505,-75.155564,MTSW4McQd7CbVtyjqoe9mw,3
4,40.338183,-75.471659,mWMc6_wTdE0EUBKIGXDVfA,9


In [11]:
checkins.head()

,business_id,number_visits
0,---kPU91CF4Lq2-WlRu9Lw,11
1,--0iUa4sNDFiZFrAdIWhZQ,10
2,--30_8IhuyMHbSOcNWd6DQ,2
3,--7PUidqRWpRSpXebiyxTg,10
4,--7jw19RH9JKXgFohspgQw,26


In [13]:
top_tips.head()

,number_tips,business_id
0,2571,FEXhWNCMkv22qG04E83Qjg
1,1011,-QI8Qi8XWH3D8y8ethnajA
2,932,_ab50qdWOk0DdB6XOrBitw
3,827,ytynqOUb3hjKeJfRj5Tshw
4,826,Eb1XmmLWyt_way5NNZ7-Pw


In [16]:
metrics.head()

,Unnamed: 0,business_id,review_count,avg(reactions),avg(stars),avg(Influencer),sum(total)
0,0,OJpwmYvsZnXt62sxco3F0Q,399,2.576642,3.822384,0.576839,153714
1,1,oZAdXhal_EZHePbjxo6s9g,15,2.866667,3.666667,0.217222,90
2,2,bvkZndsHPy0nwpn3_iKCQQ,99,1.519608,3.558824,0.856587,17544
3,3,DmkUXt42gLCFsQh_MVsAqw,60,1.661290,3.467742,0.728788,4960
4,4,wGNNCB-EjhTby2BD2iqJCw,18,1.555556,2.888889,0.668744,1242


In [17]:
target.head()

,Unnamed: 0,business_id,review_count,avg(reactions),avg(stars),avg(Influencer),success_score,target
0,0,OJpwmYvsZnXt62sxco3F0Q,399,2.576642,3.822384,0.576839,2266.813432,1
1,1,oZAdXhal_EZHePbjxo6s9g,15,2.866667,3.666667,0.217222,34.248704,0
2,2,bvkZndsHPy0nwpn3_iKCQQ,99,1.519608,3.558824,0.856587,458.611143,1
3,3,DmkUXt42gLCFsQh_MVsAqw,60,1.661290,3.467742,0.728788,251.909527,1
4,4,wGNNCB-EjhTby2BD2iqJCw,18,1.555556,2.888889,0.668744,54.093995,0


In [233]:
business2['categories'].str.split(', ', expand=True).stack().value_counts().head(10).index.tolist()

['Restaurants',
 'Food',
 'Shopping',
 'Home Services',
 'Beauty & Spas',
 'Nightlife',
 'Health & Medical',
 'Local Services',
 'Bars',
 'Automotive']

In [246]:
categories = business2['categories'].str.split(', ', expand=True)

In [247]:
categories.index = business2['business_id']

In [248]:
df = categories.T.stack().groupby('business_id').apply(list).reset_index(name='categories')

In [249]:
df

,business_id,categories
0,Pns2l4eNsfO8kk83dixA6A,"[Doctors, Traditional Chinese Medicine, Naturo..."
1,mpf3x-BjTdTEA3yCZrAYPw,"[Shipping Centers, Local Services, Notaries, M..."
2,tUFrWirKiKi_TAnsVWINQQ,"[Department Stores, Shopping, Fashion, Home & ..."
3,MTSW4McQd7CbVtyjqoe9mw,"[Restaurants, Food, Bubble Tea, Coffee & Tea, ..."
4,mWMc6_wTdE0EUBKIGXDVfA,"[Brewpubs, Breweries, Food]"
...,...,...
150238,IUQopTMmYQG-qRtBk-8QnA,"[Nail Salons, Beauty & Spas]"
150239,c8GjPIOTGVmIemT7j5_SyQ,"[Pets, Nurseries & Gardening, Pet Stores, Hobb..."
150240,_QAMST-NrQobXduilWEqSw,"[Shopping, Jewelry, Piercing, Toy Stores, Beau..."
150241,mtGm22y5c2UHNXDFAjaPNw,"[Fitness/Exercise Equipment, Eyewear & Opticia..."


In [250]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()
df = df.join(pd.DataFrame(mlb.fit_transform(df.pop('categories')),
                          columns=mlb.classes_,
                          index=df.index))

In [253]:
df['total_categories'] = df.sum(axis=1)

/tmp/ipykernel_32/3241133823.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df['total_categories'] = df.sum(axis=1)


In [255]:
most_frequent = ['Restaurants',
 'Food',
 'Shopping',
 'Home Services',
 'Beauty & Spas',
 'Nightlife',
 'Health & Medical',
 'Local Services',
 'Bars',
 'Automotive', 'total_categories', 'business_id']

In [257]:
df[most_frequent].sort_values(by='total_categories', ascending=False).head(10)

,Restaurants,Food,Shopping,Home Services,Beauty & Spas,Nightlife,Health & Medical,Local Services,Bars,Automotive,total_categories,business_id
143059,1,1,1,0,0,0,0,0,0,0,36,ytynqOUb3hjKeJfRj5Tshw
86993,0,0,1,0,1,1,0,1,1,0,26,TfRyyMfqDxtC1KA1jLkmvg
106365,1,1,0,0,0,1,0,0,1,0,25,5hyIty7KkRfVE_1gCKbUvw
86376,0,0,1,1,0,0,0,1,0,0,24,mmlZz6ypFdgHROjt2AnSsw
3086,1,0,1,0,0,0,1,1,0,1,23,kdboJGRQZXuKYLIFMvsoJw
136920,0,0,1,1,0,0,0,1,0,1,23,Zk4aemT88EwIvoa-19z4mg
57574,1,1,1,0,0,1,0,0,1,0,22,iOFbMitGdfU6i5xm4nZ8Ww
100045,1,1,0,0,0,1,0,0,1,0,22,4fKGv-3XMlxv_vuiFbuwLA
102462,1,1,0,0,0,1,0,0,0,0,22,lPday_z2XpRJqbFWyCh6dw
109339,1,1,1,0,0,0,0,0,0,0,21,-mDAFyX5kZHXhxjELeMhiQ


In [259]:
df[most_frequent].to_csv('./data/categories_fixed.csv', index=False)